## Step 1: Install Dependencies


In [ ]:
# Install all required packages
!pip install -q langchain>=0.1.0 langchain-openai>=0.0.2 langchain-community>=0.0.10 langgraph>=0.0.20
!pip install -q unstructured[pdf] pypdf pdf2image Pillow
!pip install -q chromadb faiss-cpu
!pip install -q sentence-transformers torch torchvision
!pip install -q duckduckgo-search tavily-python
!pip install -q python-dotenv requests opencv-python
!pip install -q numpy==1.24.3 pydantic>=2.7.4,<3.0.0

# Install system dependencies for PDF processing
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr

print("✅ All dependencies installed!")


## Step 2: Set API Keys


In [ ]:
import os

# Set your API keys here
HUGGINGFACE_API_KEY = "YOUR_HUGGINGFACE_API_KEY_HERE"  # Required
TAVILY_API_KEY = "YOUR_TAVILY_API_KEY_HERE"  # Optional, for web search

# Set environment variables
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["USE_HUGGINGFACE_PRIMARY"] = "true"
os.environ["USE_OPENAI_EMBEDDINGS"] = "false"
os.environ["USE_OPENAI_FALLBACK"] = "false"

# Fix for protobuf compatibility
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

print("✅ API keys configured!")


## Step 3: Upload Project Files

Upload all the Python files from the project. You can either:
1. Use the file uploader below
2. Or clone from GitHub if you have the repo


In [ ]:
# Upload project files
from google.colab import files
import zipfile
import os

# Upload a zip file containing all project files, or upload individual .py files
uploaded = files.upload()

# Extract if it's a zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✅ Extracted {filename}")
    else:
        print(f"✅ Uploaded {filename}")

# List uploaded files
print("\n📁 Project files:")
for f in os.listdir('.'):
    if f.endswith('.py'):
        print(f"  - {f}")


## Step 4: Initialize the System


In [ ]:
# Import all modules
from pdf_processor import MultimodalPDFProcessor
from vector_store import MultimodalVectorStore
from agents import MultiAgentRAG
import config

# Initialize vector store
print("📦 Initializing vector store...")
vector_store = MultimodalVectorStore()
print(f"✅ Vector store initialized with: {vector_store.embedding_type} embeddings")

# Initialize RAG system
print("🤖 Initializing RAG system...")
rag_system = MultiAgentRAG(vector_store, use_huggingface_primary=True)
print(f"✅ RAG system initialized with: {rag_system.primary_llm_type} LLM")

print("\n🎉 System ready!")


## Step 5: Upload and Process PDF


In [ ]:
from google.colab import files
import tempfile
import os

# Upload PDF file
uploaded = files.upload()

# Process PDF
processor = MultimodalPDFProcessor(processing_mode=config.PDF_PROCESSING_MODE)

all_chunks = []
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        print(f"\n📄 Processing {filename}...")
        chunks = processor.process_pdf(filename)
        all_chunks.extend(chunks)
        print(f"✅ Extracted {len(chunks)} chunks from {filename}")

# Add to vector store
if all_chunks:
    print(f"\n💾 Adding {len(all_chunks)} chunks to vector store...")
    vector_store.add_documents(all_chunks)
    print("✅ Documents added successfully!")
else:
    print("⚠️ No chunks extracted from PDF")


## Step 6: Ask Questions


In [ ]:
# Ask a question about your documents
question = "tell me about the image in doc"  # Change this to your question

print(f"❓ Question: {question}\n")
print("🤔 Thinking...\n")

try:
    answer = rag_system.query(question)
    print(f"\n💬 Answer:\n{answer}")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
